# Knapsack problem

Given a list of items and a bag maximum capacity, find the best combination of items.

The algorithm to complete:

In [91]:
import random
import math
import functools 

class KnapsackGenetic:
  def __init__(self, params):
    self.ALL_NUMBERS = list(range(params["max_per_item"] + 1))
    self.params = params
    self.specimen = [None] * self.params["generation_size"]

    self.create_initial_population()

  def create_initial_population(self):
    self.specimen = list(map(
        lambda _: list(map(
            lambda _: random.choice(self.ALL_NUMBERS),
            [None] * len(self.params["items"])
        )),
        self.specimen
    ))

  def fitness(self, specimen):
    val = 0
    wgth = 0
    #maximo valor posible para obtener sin restricion de peso.
    max_value = sum(map(lambda x: x.value*self.params["max_per_item"], self.params["items"]))
    for i in range(len(specimen)):
      val += specimen[i]*self.params["items"][i].value
      wgth += specimen[i]*self.params["items"][i].weight
    return 0 if wgth > self.params["max_weight"] else val / max_value

  def is_converged(self):
    if any(self.fitness(specimen) >= self.params["fit_threshold"] for specimen in self.specimen):
      return True

    return False

  def get_fit(self):
    evaluations = self.fitness_all()

    max_evaluation = max(evaluations)

    max_index = evaluations.index(max_evaluation)

    return self.specimen[max_index], max_evaluation

  def fitness_all(self):
    return list(map(self.fitness, self.specimen))

  def select_specimen(self, specimen_evaluations):
    specimen_and_evaluations = list(zip(self.specimen, specimen_evaluations))

    specimen_and_evaluations.sort(key=lambda e: e[1], reverse = True)

    n_top = int(math.ceil(len(self.specimen) * params["select_top"]))

    return list(map(lambda s: s[0], specimen_and_evaluations[:n_top]))
  
  def mutate(self, specimen):
    n_digits = int(params["mutation_percentage"] * (len(specimen)))

    digit_indexes = random.sample(list(range(len(specimen))), n_digits)

    mutated = specimen[:]

    for idx in digit_indexes:
      mutated[idx] = random.randint(0, self.params["max_per_item"])

    return mutated
    
  def generate_children(self, selected_specimen):  
    mutated_specimen = [None] * len(self.specimen)

    for i in range(len(mutated_specimen)):
      mutated_specimen[i] = self.mutate(random.choice(selected_specimen))

    return mutated_specimen

  def run(self):
    generation_number = 1
    
    best = None
    max_eval = -0xffffff

    while generation_number <= self.params["max_generations"] and not self.is_converged():
      top_generation = self.get_fit()
      top_str = "".join(str(top_generation[0]))
      
      if(max_eval < top_generation[1]):
        best = top_generation[0]
        max_eval = top_generation[1]

      print(f"Generation #{generation_number}:\t{top_str}\t{top_generation[1]}")

      specimen_evaluations = self.fitness_all()
      selected_specimen = self.select_specimen(specimen_evaluations)
      
      self.specimen = self.generate_children(selected_specimen)
      
      generation_number += 1
    

    print("Best: {0}, {1}".format(best, max_eval))
    return self.get_fit()

To run the algorithm:

In [92]:
class Item:
  def __init__(self, value, weight):
    self.value = value
    self.weight = weight

params = {
    "mutation_percentage": 1,
    "select_top": 0.05,
    "generation_size": 100,
    "fit_threshold": 0.8,
    "max_generations": 100,
    "max_weight": 15,
    "max_per_item": 5,
    "items": [Item(4, 12), Item(2, 2), Item(2, 1), Item(1, 1), Item(10,4)]
}

knapsack = KnapsackGenetic(params)
knapsack.run()


Generation #1:	[0, 0, 3, 0, 3]	0.37894736842105264
Generation #2:	[0, 0, 5, 0, 1]	0.21052631578947367
Generation #3:	[0, 0, 1, 1, 3]	0.3473684210526316
Generation #4:	[0, 0, 0, 3, 3]	0.3473684210526316
Generation #5:	[0, 0, 5, 4, 1]	0.25263157894736843
Generation #6:	[0, 1, 0, 0, 3]	0.3368421052631579
Generation #7:	[0, 0, 0, 0, 3]	0.3157894736842105
Generation #8:	[0, 4, 5, 0, 0]	0.18947368421052632
Generation #9:	[0, 1, 4, 0, 2]	0.3157894736842105
Generation #10:	[0, 1, 0, 3, 2]	0.2631578947368421
Generation #11:	[0, 1, 0, 1, 2]	0.24210526315789474
Generation #12:	[0, 0, 2, 4, 2]	0.29473684210526313
Generation #13:	[0, 3, 0, 3, 1]	0.2
Generation #14:	[0, 1, 4, 1, 2]	0.3263157894736842
Generation #15:	[0, 2, 2, 1, 2]	0.30526315789473685
Generation #16:	[0, 4, 1, 0, 1]	0.21052631578947367
Generation #17:	[0, 0, 4, 0, 2]	0.29473684210526313
Generation #18:	[0, 2, 0, 0, 2]	0.25263157894736843
Generation #19:	[0, 1, 5, 0, 2]	0.3368421052631579
Generation #20:	[0, 0, 3, 2, 2]	0.29473684210

([0, 2, 0, 1, 2], 0.2631578947368421)